In [34]:
from pathlib import Path
from dclpatch.main import PatchProject


SyntaxError: invalid syntax (database.py, line 317)

In [ ]:
root_dir_path = Path('/mnt/c/Users/dsege/TEMP/Alexia/analysis_with_dcl_patch/')

In [ ]:
api = PatchProject(root_dir = root_dir_path)

In [ ]:
api.load_database()

In [ ]:
path_to_cell_recordings_dir = Path('/mnt/c/Users/dsege/TEMP/Alexia/NET-cre_vGat-flp/2021_12_09_C2/')

In [ ]:
api.add_cell_to_database(path_to_cell_recordings_dir = path_to_cell_recordings_dir, overwrite=True)

In [ ]:
path_to_all_cell_recordings = Path('/mnt/c/Users/dsege/TEMP/Alexia/NET-cre_vGat-flp/')
api.add_all_cells_to_database(path_with_cell_recordings_as_subdirs = path_to_all_cell_recordings, overwrite=True)

In [ ]:
api.database.cell_recordings_metadata

In [ ]:
api.compare_on_single_cell_level(global_cell_id = '0000')

In [ ]:
api.compare_within_group(group_column = 'brain_region', group_id = 'vlPAG')

In [ ]:
api.save_database()

# Code development

In [133]:
import pandas as pd
import numpy as np
from pathlib import Path

In [134]:
path = Path('/mnt/c/Users/dsege/TEMP/Alexia/NET-cre_vGat-flp/2021_12_09_C2/')

In [125]:
test = pd.read_excel('/mnt/c/Users/dsege/TEMP/Alexia/NET-cre_vGat-flp/2021_12_09_C2/2021_12_09_C2.xlsx', sheet_name = None)

In [130]:
input_df = test['Recordings and cell properties']

In [ ]:
'stimulation_string', 'stimulation_frequency-Hz', 'stimulation_duration-ms', 'filepath_detected_events'

In [131]:
input_df.columns = input_df.iloc[1]
input_df = input_df.iloc[2:, 1:]
input_df = input_df.loc[input_df['Type of protocol'].notnull()]

columns = list(input_df.columns)
relevant_data_columns = columns[columns.index('Pharmacology') +1 :]

all_final_single_stimulation_dfs = []
for row_index in range(input_df.shape[0]):
    single_stimulation_df = input_df.iloc[[row_index]].copy()
    if type(single_stimulation_df['Time since cutting'].iloc[0]) == datetime.time:
        time_since_cutting = single_stimulation_df['Time since cutting'].iloc[0].hour * 60 + single_stimulation_df['Time since cutting'].iloc[0].minute
    else:
        time_since_cutting = 'not_available'

    if single_stimulation_df['Vh (mV)'].iloc[0] == -70:
        postsynaptic_current_type = 'excitatory'
    elif single_stimulation_df['Vh (mV)'].iloc[0] == 0:
        postsynaptic_current_type = 'inhibitory'
    else:
        raise ValueError(f'{single_stimulation_df["Vh (mV)"].iloc[0]} is not a valid entry for holding potential!')

    if pd.isna(single_stimulation_df['Pharmacology'].iloc[0]):
        pharmacology = 'none'
    else:
        pharmacology = single_stimulation_df['Pharmacology'].iloc[0]

    if single_stimulation_df['Type of protocol'].iloc[0] == 'Bsl':
        stimulation_frequency = np.NaN
        stimulation_duration = np.NaN
        stimulation_string = 'baseline'
        # filepath?
    elif single_stimulation_df['Type of protocol'].iloc[0] == 'Bsl2':
        stimulation_frequency = np.NaN
        stimulation_duration = np.NaN
        stimulation_string = 'control_baseline'
        # filepath?
    elif 'Hz' in single_stimulation_df['Type of protocol'].iloc[0]:
        stimulation_frequency = single_stimulation_df['Type of protocol'].iloc[0].replace(' Hz', '')
        stimulation_duration = int(single_stimulation_df['Timing of stim (s)'].iloc[0] * 1000)
        stimulation_string = f'{stimulation_frequency}-Hz_for_{stimulation_duration}-ms'
        # filepath


    sorted_data = {'postsynaptic_current_type': [postsynaptic_current_type],
                   'stimulation_string': [stimulation_string],
                   'stimulation_frequency-Hz': [stimulation_frequency],
                   'stimulation_duration-ms': [stimulation_duration],
                   'pharmacology': [pharmacology],
                   'time_since_cutting-M': [time_since_cutting]}

    sorted_single_stimulation_df = pd.DataFrame(data = sorted_data)
    remaining_data_df = single_stimulation_df[relevant_data_columns].copy()
    remaining_data_df = remaining_data_df.reset_index(drop=True)
    all_final_single_stimulation_dfs.append(pd.concat([sorted_single_stimulation_df, remaining_data_df], axis=1))
recording_specific_dataframe = pd.concat(all_final_single_stimulation_dfs)
final_df = pd.concat([general_metadata_df, recording_specific_dataframe], axis=1)
final_df.insert(0, 'global_cell_id', [global_cell_id]*final_df.shape[0])

In [136]:
metadata_df = test['General information']
general_metadata = {'date': path.name[:10],
                    'session_cell_id': path.name[path.name.rfind('_') + 1:],
                    'mouse_line': metadata_df['Animal line'][0],
                    'brain_region': metadata_df['Region'][0],
                    'cell_type': metadata_df['Type'][0],
                    'sex': metadata_df['Sex'][0],
                    'age': metadata_df['Age (days)'][0],
                    'stimulation_in_percent': metadata_df['Stimulation (%)'][0],
                    'internal_patch_solution': metadata_df['Internal used'][0]}
general_metadata_df = pd.DataFrame(general_metadata, index=[0])

In [139]:
global_cell_id = '0000'

In [140]:
final_df = pd.concat([general_metadata_df, recording_specific_dataframe], axis=1)
final_df.insert(0, 'global_cell_id', [global_cell_id]*final_df.shape[0])

In [145]:
a = {'hund': 'dog', 
     'katze': 'cat'}

for key, value in a.items():
    print(key, value)

hund dog
katze cat


In [144]:
pd.concat([final_df, final_df])

,global_cell_id,date,session_cell_id,mouse_line,brain_region,cell_type,sex,age,stimulation_in_percent,internal_patch_solution,...,Iinf(nA),Rm (MΩ),Access change (%),Total time (s),Time of counting (s),Nb of sweeps,Mean amp (pA),Rise time 10-90 (ms),Decay time 63% (ms),Half width (ms)
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,5.93,4.467127,NaN,272,34,8,8.59,0.82,1.85,3.66
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,6.58,7.206247,-29.481263,313,31.3,10,8.71,1.21,1.71,3.56
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,6.71,7.273929,-29.788936,270,27,10,8.38,1.25,1.98,3.93
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,7.6,7.370798,-29.161282,321,32.1,10,8,1.3,2.17,4.12
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,11.79,7.590642,-27.08141,222.6,31.8,7,7.75,1.36,2.22,4.32
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,5.93,4.467127,NaN,272,34,8,8.59,0.82,1.85,3.66
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,6.58,7.206247,-29.481263,313,31.3,10,8.71,1.21,1.71,3.56
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,6.71,7.273929,-29.788936,270,27,10,8.38,1.25,1.98,3.93
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,7.6,7.370798,-29.161282,321,32.1,10,8,1.3,2.17,4.12
0,0000,2021_12_09,C2,NET-cre:vGat-flp,vlPAG,Unknown,Female,147,15,Kglu + 1.73ug bio,...,11.79,7.590642,-27.08141,222.6,31.8,7,7.75,1.36,2.22,4.32


,postsynaptic_current_type,stimulation_string,stimulation_frequency-Hz,stimulation_duration-ms,pharmacology,time_since_cutting-M
0,excitatory,4-Hz_for_1000-ms,4,1000,none,202


,postsynaptic_current_type,stimulation_string,stimulation_frequency-Hz,stimulation_duration-ms,pharmacology,time_since_cutting-M,Cap I (pA),Tau (ms),Rs (MΩ),Cell Cap (pF),Iinf(nA),Rm (MΩ),Access change (%),Total time (s),Time of counting (s),Nb of sweeps,Mean amp (pA),Rise time 10-90 (ms),Decay time 63% (ms),Half width (ms)
0,excitatory,4-Hz_for_1000-ms,4,1000,none,202,-114.6,1.74,8.726003,199.404,6.58,7.206247,-29.481263,313,31.3,10,8.71,1.21,1.71,3.56


In [71]:
time_since_cutting = single_stimulation_df['Time since cutting'].iloc[0].hour * 60 + single_stimulation_df['Time since cutting'].iloc[0].minute

In [114]:

remaining_data_df

1,Cap I (pA),Tau (ms),Rs (MΩ),Cell Cap (pF),Iinf(nA),Rm (MΩ),Access change (%),Total time (s),Time of counting (s),Nb of sweeps,Mean amp (pA),Rise time 10-90 (ms),Decay time 63% (ms),Half width (ms)
0,-114.6,1.74,8.726003,199.404,6.58,7.206247,-29.481263,313,31.3,10,8.71,1.21,1.71,3.56


In [112]:
single_stimulation_df[data_columns]

1,Cap I (pA),Tau (ms),Rs (MΩ),Cell Cap (pF),Iinf(nA),Rm (MΩ),Access change (%),Total time (s),Time of counting (s),Nb of sweeps,Mean amp (pA),Rise time 10-90 (ms),Decay time 63% (ms),Half width (ms)
3,-114.6,1.74,8.726003,199.404,6.58,7.206247,-29.481263,313,31.3,10,8.71,1.21,1.71,3.56


In [75]:
import datetime
isinstance(time_since_cutting, datetime.time)

False

In [47]:
tmp_str = tmp_str.replace(' Hz', '')
tmp_str

'4'

In [ ]:
'stimulation_string', 'stimulation_frequency-Hz', 'stimulation_duration-ms', 'filepath_detected_events'

In [ ]:
database = api.database

In [ ]:
database.__dict__.keys()

In [ ]:
database.subdirectories

In [ ]:
df = api.database.cell_recordings_metadata

In [ ]:
df['sex'].unique().shape[0]

In [ ]:
list(df.loc[df['sex'] == 'Female', 'global_cell_id'].unique())

In [ ]:
import os

In [ ]:
os.path.isdir('/mnt/c/Users/dsege/TEMP/Alexia/analysis_with_dcl_patch/')

In [ ]:
p = Path('/mnt/c/Users/dsege/TEMP/Alexia/analysis_with_dcl_patch/')

In [ ]:
p.joinpath('results/')

In [ ]:
import os
from pathlib import Path
from typing import Dict

import pandas as pd
import numpy as np

In [ ]:
def get_general_metadata_df(path_to_recordings_dir: Path) -> pd.DataFrame:
    metadata_df = pd.read_excel(path_to_recordings_dir.joinpath(f'{path_to_recordings_dir.name}.xlsx'),
                                sheet_name = 'General information')
    general_metadata = {'date': path_to_recordings_dir.name[:10],
                        'session_cell_id': path_to_recordings_dir.name[path_to_recordings_dir.name.rfind('_') + 1:],
                        'mouse_line': metadata_df['Animal line'][0],
                        'brain_region': metadata_df['Region'][0],
                        'cell_type': metadata_df['Type'][0],
                        'sex': metadata_df['Sex'][0]}
    return pd.DataFrame(general_metadata, index=[0])

In [ ]:
def get_stimulation_paradigms_df(path_to_recordings_dir: Path) -> pd.DataFrame:
    filepaths_stimulation_recordings = list()
    for elem in path_to_recordings_dir.iterdir():
        if 'datapoints' not in elem.name:
            if elem.name.endswith('.csv'):
                filepaths_stimulation_recordings.append(elem)
    stimulation_paradigms = {'stimulation_string': list(),
                             'stimulation_frequency-Hz': list(),
                             'stimulation_duration-ms': list(), 
                             'filepath_detected_events': list()}
    for filepath in filepaths_stimulation_recordings:
        filename = filepath.name
        filename = filename.replace('.csv', '')
        yyyy_mm_dd_ = filename[:11]
        filename = filename.replace(yyyy_mm_dd_, '')
        cell_id = filename[:filename.index('_')]
        filename = filename.replace(cell_id + '_', '')
        if 'Hz' in filename:
            stimulation_frequency = int(filename[:filename.index('Hz')])
            stimulation_duration = filename[filename.index('Hz') + 2 :]
            if 'ms' not in stimulation_duration:
                stimulation_duration = int(stimulation_duration[:stimulation_duration.find('s')]) * 1000
            else:
                stimulation_duration = int(stimulation_duration[:stimulation_duration.find('ms')])
            stimulation_paradigm = f'{stimulation_frequency}-Hz_for_{stimulation_duration}-ms'
        elif 'Bsl' in filename:
            stimulation_frequency = np.NaN
            stimulation_duration = np.NaN
            stimulation_paradigm = 'baseline'
        else:
            print(f'Warning: stimulation paradigm could not be identified for: {filepath.name}')
            stimulation_frequency = np.NaN
            stimulation_duration = np.NaN
            stimulation_paradigm = 'unknown'
        stimulation_paradigms['stimulation_string'].append(stimulation_paradigm)
        stimulation_paradigms['stimulation_frequency-Hz'].append(stimulation_frequency)
        stimulation_paradigms['stimulation_duration-ms'].append(stimulation_duration)
        stimulation_paradigms['filepath_detected_events'].append(filepath)
    return pd.DataFrame(stimulation_paradigms)

In [ ]:
global_cell_id = 1

In [ ]:
str(global_cell_id).zfill(4)

In [ ]:
def get
    general_metadata = get_general_metadata_df(cell_recording_dir)
    stimulation_paradigms = get_stimulation_paradigms_df(cell_recording_dir)
    stimulations_adjusted_general_metadata = pd.concat([general_metadata]*stimulation_paradigms.shape[0], ignore_index=True)
    recordings_overview = pd.concat([stimulations_adjusted_general_metadata, stimulation_paradigms], axis=1)
    recordings_overview['global_cell_id'] = str(global_cell_id).zfill(4)


In [ ]:
columns = ['global_cell_id', 'date', 'session_cell_id', 'mouse_line', 'sex', 'brain_region', 'cell_type', 
           'stimulation_string', 'stimulation_frequency-Hz', 'stimulation_duration-ms', 'filepath_detected_events']

In [ ]:
recordings_overview[columns]

In [ ]:
recordings_overview

In [ ]:
stimulation_paradigms

In [ ]:
df_cell = pd.DataFrame(recording_info)

In [ ]:
df_cell

In [ ]:
cell_recording_dir = Path('/mnt/c/Users/dsege/TEMP/Alexia/NET-cre_vGat-flp/2021_12_09_C2/')



metadata_df = pd.read_excel(cell_recording_dir.joinpath(f'{cell_recording_dir.name}.xlsx'),
                            sheet_name = 'General information')

mix_and_matchable_categories = dict()

general_metadata = {'mouse_line': metadata_df['Animal line'][0],
                    'brain_region': metadata_df['Region'][0],
                    'cell_type': metadata_df['Type'][0],
                    'sex': metadata_df['Sex'][0]}

filepaths_stimulation_recordings = list()
for elem in cell_recording_dir.iterdir():
    if 'datapoints' not in elem.name:
        if elem.name.endswith('.csv'):
            filepaths_stimulation_recordings.append(elem)


stimulation_paradigms = dict()
for filepath in filepaths_stimulation_recordings:
    filename = filepath.name
    filename = filename.replace('.csv', '')
    yyyy_mm_dd_ = filename[:11]
    filename = filename.replace(yyyy_mm_dd_, '')
    cell_id = filename[:filename.index('_')]
    filename = filename.replace(cell_id + '_', '')
    if 'Hz' in filename:
        stimulation_frequency = filename[:filename.index('Hz') + 2]
        stimulation_duration = filename[filename.index('Hz') + 2 :]
        stimulation_paradigm = f'{stimulation_frequency}_for_{stimulation_duration}'
    elif 'Bsl' in filename:
        stimulation_paradigm = 'baseline'
    else:
        print(f'Warning: stimulation paradigm could not be identified for: {filepath.name}')
        stimulation_paradigm = 'unknown'
    stimulation_paradigms[stimulation_paradigm] = filepath

In [ ]:
filepaths_stimulation_recordings

In [ ]:
stimulation_paradigms

In [ ]:
?filename.replace

In [ ]:
metadata_df['Animal line'][0]

In [ ]:
metadata_df

In [ ]:
df = pd.read_excel(cell_recording_dir.as_posix() + '/' + cell_recording_dir.name + '.xlsx', sheet_name = 'General information')
df

In [ ]:
df = pd.read_excel(cell_recording_dir.as_posix() + '/' + cell_recording_dir.name + '.xlsx', sheet_name = 'Recordings and cell properties', skiprows=2)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.columns

In [ ]:
dfs['General information']

In [ ]:
dfs['Recordings and cell properties']

In [ ]:
for elem in cell_recording_dir.iterdir():
    print(elem)

In [ ]:
pd.read_csv(filepath)

In [ ]:
path = Path('/mnt/c/Users/dsege/TEMP/Alexia/')

In [ ]:
for elem in path.iterdir():
    print(elem)

In [ ]:
for elem in path.glob(pattern='*'):
    print(elem)

In [ ]:
path.parts

In [ ]:
database = Database(path)

In [ ]:
import os
from pathlib import Path
from abc import ABC, abstractmethod

class Feature(ABC):
    
    @abstractmethod
    def add_to_database(self):
        pass
    

class CellRecording(Feature):
    
    def add_to_database(self):
        pass
    
    
class Subdirectories:
    
    def __init__(self, root_dir: Path):
        self.root_dir = root_dir
        self.create_missing_subdirectories()
        self.assign_subdirectories_as_attributes()
        
    def create_missing_subdirectories(self):
        # check for each element in a list of subdirs, whether they exist --> create if not
        pass
    
    def assign_subdirectories_as_attributes(self):
        # use list of subdirs and set the path to each as attribute
        pass   


class Database:
    
    """
    The database is supposed to hold all information about all recorded cells that were added to it.
    These information exclude the raw data (only contain filepaths to raw data),
    but include general metadata like celltype, stimulus type, brain region, pharamacological treatment.
    Potentially, some intermediate data could be added (like how the cell reacted upon stimulation, see dashboard).
    Overall, the database should allow the selection of cells based on mix-and-match criteria 
    for further (statistical) analyses and plotting.
    """
    
    def __init__(self, root_dir: Path):
        self.root_dir = root_dir
        self.subdirectories = Subdirectories(root_dir = root_dir)

    def load_database_from_disk(self):
        # Option to load previously created and saved database
        pass
    
    def save_database_to_disk(self):
        # Save all information to disk
        pass
    
    def add_new_feature(self, new_feature: Feature):
        # By providing the Path to a new directory, the user has the option to add new recordings to the database
        # should trigger the database to update itself (e.g. categories dictionary)
        pass


    

        
